In [31]:
#フォロワー・フォロー
# !pip install tweepy

In [18]:
import json
import tweepy
from datetime import datetime,timezone
import pytz
import pandas as pd
import datetime

In [19]:
username = "git_media"

In [20]:
f = open('secret.json','r')
key = json.load(f)
Consumer_key = key['Consumer_key']
Consumer_secret = key['Consumer_secret']
Access_token = key['Access_token']
Access_secret = key['Access_secret']
bearer_token = key['bearer_token']

In [21]:
def get_userid(username):
    username = username
    user_result = client.get_user(username=username)
    id = user_result.data.id
    return id
def change_time_JST(u_time):
    tm = u_time.astimezone(pytz.timezone('Asia/Tokyo'))
    zn = tm.strftime('%Y/%m/%d %H:%M:%S')
    return zn
def csvize(tw_data,text,labels,name):
    name = name
    #tw_dataのリストをpandasのDataFrameに変換
    df = pd.DataFrame(tw_data,columns=labels)
    #CSVファイルの名前を決める
    file_name=name + '_'+text+'_'+time_now+'.csv'
    #CSVファイルを出力する
    df.to_csv('file/'+file_name,encoding='utf-8-sig',index=False)
    
dt_now = datetime.datetime.now()
time_now = dt_now.strftime('%Y%m%d%H%M%S')

In [22]:
# OAuth認証
auth = tweepy.OAuthHandler(Consumer_key, Consumer_secret)
auth.set_access_token(Access_token, Access_secret)
api = tweepy.API(auth)

client = tweepy.Client(
    consumer_key=Consumer_key,
    consumer_secret=Consumer_secret,
    access_token=Access_token,
    access_token_secret=Access_secret,
    bearer_token=bearer_token
)

In [23]:
user_id = get_userid(username)
labels=['アカウントID','アカウント名']

In [24]:
tw_data = []
next_token = ""
first_flg = True
while True:
    print("first_flg: " + str(first_flg))
    if first_flg == True:
        #1回目リクエスト    
        follows = client.get_users_following(user_id)
        first_flg = False
    else:
        #2回目リクエスト    
        follows = client.get_users_following(user_id,pagination_token=next_token)
    for user in follows.data:
        tw_data.append([
        user.username, user.name]) 
    try:
        next_token = follows[3]['next_token']
        print("next_token: " + next_token)
    except KeyError:
        #next_tokenが取得できない（最後のリクエスト）の場合はループ終了
        print("最後のページに到達しました")
        break
print(f"取得したデータ数：{len(tw_data)}")

first_flg: True
next_token: RGMN40TL338HEZZZ
first_flg: False
next_token: AVBTU921EV81EZZZ
first_flg: False
next_token: 600BCD0LTN5HEZZZ
first_flg: False
next_token: U4I5J9SQVV11EZZZ
first_flg: False
next_token: 6GB3ML0DSAV1EZZZ
first_flg: False
最後のページに到達しました


In [25]:
csvize(tw_data,username,labels,"follows")

In [26]:
tw_data = []
next_token = ""
first_flg = True
while True:
    print("first_flg: " + str(first_flg))
    if first_flg == True:
        #1回目リクエスト
        followers = client.get_users_followers(id=user_id, max_results="100")
        #初回フラグを落とす
        first_flg = False
    else:
        #2回目移行リクエスト
        followers = client.get_users_followers(id=user_id, max_results="100", pagination_token=next_token)
    for user in followers.data:
        tw_data.append([
        user.username, user.name])
    print(len(tw_data))
        
    try:
        next_token = followers.meta['next_token']
        print("next_token: " + next_token)
    except KeyError:
        #next_tokenが取得できない（最後のリクエスト）の場合はループ終了
        print("最後のページに到達しました")
        break
print(f"取得したデータ数：{len(tw_data)}")

first_flg: True
100
next_token: AKFU49VINR81EZZZ
first_flg: False
200
next_token: 191CJCIPHN31EZZZ
first_flg: False
300
next_token: NCQHDDUOM701EZZZ
first_flg: False
400
next_token: 7S2NKQV362V1EZZZ
first_flg: False
425
最後のページに到達しました


In [16]:
csvize(tw_data,username,labels,"follower")

In [ ]:
#まだ未完成

In [11]:
# フォローユーザー分だけループ
for f in follows:
 
    # フォローユーザーがフォロワー一覧に含まれていないなら
    if f not in followers:
        print("ID:{}、ユーザー名:{}をアンフォローしますか？".format(api.get_user(f).screen_name, api.get_user(f).name))
        if input("Y/N?") in ['y','Y']:    
            api.destroy_friendship(f)
            print("ID:{}、ユーザー名:{}をフォローを解除しました。".format(api.get_user(f).screen_name, api.get_user(f).name))

TypeError: get_user() takes 1 positional argument but 2 were given